In [1]:
# 동적 크롤링 라이브러리(selenium) 설치
!pip install selenium

In [2]:
# selnium 버전 업그레이드
!pip install selenium --upgrade

In [3]:
# 자동으로 웹 드라이버 불러오는 명령어 
!pip install webdriver-manager

In [4]:
# 동적 크롤링 라이브러리 세팅
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys # import할 때는 k가 대문자 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

In [19]:
# 완성된 동적 크롤링 세팅 최종 코드 (성균관대학교 자연과학캠퍼스 주변 식당)
options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)

service = Service(ChromeDriverManager().install())

driver = wb.Chrome(service=service, options=options)
time.sleep(1)

url = "https://map.naver.com/p"
driver.get(url)
time.sleep(1)

# 검색어 입력
search_box = driver.find_element(By.CSS_SELECTOR, "input.input_search")
search_box.clear()
search_box.send_keys("성균관대학교 자연과학캠퍼스 맛집")
search_box.send_keys(Keys.ENTER)
time.sleep(1.5)

# 데이터를 담기 위한 빈 리스트 생성
NameList = []
NametypeList = []
LocationList = []
LocinfoList = []
InfoList = []
MenuList = []
MenuinfoList = []
PriceList = []

TOTAL_LIMIT = float('inf')
count_saved = 0
visited_ids = set()


# ============================================================
# 🔥 1. 스크롤 함수
# ============================================================
def scroll_page_with_pagedown(driver, max_scroll=200):
    """
    PageDown을 이용해 천천히 전체 목록을 끝까지 스크롤하는 함수.
    max_scroll: 페이지 다운 횟수 (200~300 정도면 충분)
    """

    driver.switch_to.default_content()
    time.sleep(1)

    # 검색 iframe 진입
    search_iframe = driver.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
    driver.switch_to.frame(search_iframe)

    scroll_box = driver.find_element(By.CSS_SELECTOR, "div#_pcmap_list_scroll_container")

    print("📌 PageDown 방식으로 전체 스크롤 시작…")

    last_items_count = -1
    stable_count = 0

    for i in range(max_scroll):

        # PageDown 키보드 입력
        scroll_box.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)

        # 현재 로딩된 item 수 체크
        items = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
        current_count = len(items)

        # 더 이상 새로운 가게가 안 나오면 멈춤
        if current_count == last_items_count:
            stable_count += 1
            if stable_count >= 5:
                print("📌 더 이상 새로운 가게가 로딩되지 않아 스크롤 중단")
                break
        else:
            stable_count = 0

        last_items_count = current_count

    print(f"📌 PageDown 스크롤 완료. 총 로딩된 가게 수: {last_items_count}")
    return

def get_all_items_after_full_scroll(driver):
    scroll_page_with_pagedown(driver, max_scroll=300)

    # 스크롤 끝까지 이후 가게 요소 다시 수집
    driver.switch_to.default_content()
    search_iframe = driver.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
    driver.switch_to.frame(search_iframe)

    items = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
    print(f"📌 최종 수집 가능한 가게 수: {len(items)}")
    return items


# ============================================================
# 🔥 2. 페이지 이동 함수
# ============================================================
def go_to_page(driver, page_num):
    driver.switch_to.default_content()
    search_iframe = driver.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
    driver.switch_to.frame(search_iframe)

    page_buttons = driver.find_elements(By.CSS_SELECTOR, "a.mBN2s")

    for btn in page_buttons:
        if btn.text.strip() == str(page_num):
            driver.execute_script("arguments[0].click();", btn)
            print(f"➡ {page_num} 페이지 이동")
            time.sleep(1)
            return True

    print("❌ 페이지 버튼 없음 → 마지막 페이지 도달")
    return False


# ============================================================
# 🔥 3. 상세 페이지 열기
# ============================================================
def open_store_page(driver, idx):
    items = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
    items[idx].click()
    time.sleep(1)

    driver.switch_to.default_content()

    try:
        detail_iframe = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
        )
        driver.switch_to.frame(detail_iframe)
    except:
        print("❌ iframe 실패 → 재시도")
        items = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
        items[idx].click()
        time.sleep(1)
        driver.switch_to.default_content()
        detail_iframe = driver.find_element(By.CSS_SELECTOR, "iframe#entryIframe")
        driver.switch_to.frame(detail_iframe)

    print("📌 상세 iframe 입장 완료")


# ============================================================
# 🔥 4. 메뉴 스크롤 확장
# ============================================================
def expand_menu_and_scroll(driver):
    try:
        more_btns = driver.find_elements(By.CSS_SELECTOR, "span.TeItc")
        for b in more_btns:
            if "펼쳐서 더보기" in b.text:
                driver.execute_script("arguments[0].click();", b)
                time.sleep(0.3)
    except:
        pass

    prev_h = -1
    while True:
        cur_h = driver.execute_script("return document.body.scrollHeight")
        if cur_h == prev_h:
            break
        prev_h = cur_h
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.3)

    print("📌 메뉴 전체 로딩 완료")


# ============================================================
# 🔥 6. 탭 클릭
# ============================================================
def click_tab(driver, tab_name):
    try:
        tabs = driver.find_elements(By.CSS_SELECTOR, "a[role='tab']")
        for t in tabs:
            if t.text.strip() == tab_name:
                driver.execute_script("arguments[0].click();", t)
                time.sleep(0.6)
                return True
    except:
        pass

    print(f"❌ '{tab_name}' 탭 클릭 실패")
    return False


# ============================================================
# 🔥 7. 상세 페이지 크롤링 (리뷰 제거 버전)
# ============================================================
def scrape_store_detail(driver):

    try:
        store_id = driver.find_element(
            By.CSS_SELECTOR, "meta[property='og:url']"
        ).get_attribute("content")
    except:
        store_id = None

    # ---------------- 홈 ----------------
    click_tab(driver, "홈")
    time.sleep(1)

    try: name = driver.find_element(By.CSS_SELECTOR, "span.GHAhO").text
    except: name = ""

    try: name_type = driver.find_element(By.CSS_SELECTOR, "span.lnJFt").text
    except: name_type = ""

    try: location = driver.find_element(By.CSS_SELECTOR, "span.LDgIH").text
    except: location = ""

    try:
        btn = driver.find_element(By.CSS_SELECTOR, "svg.Ky28p")
        driver.execute_script("arguments[0].click();", btn)
        time.sleep(0.2)
    except:
        pass

    try: locinfo = driver.find_element(By.CSS_SELECTOR, "span.zPfVt").text
    except: locinfo = ""

    try: info = driver.find_element(By.CSS_SELECTOR, "div.xPvPE").text
    except: info = ""

    # ---------------- 메뉴 ----------------
    click_tab(driver, "메뉴")
    time.sleep(0.7)

    driver.switch_to.default_content()
    entry_iframe = driver.find_element(By.CSS_SELECTOR, "iframe#entryIframe")
    driver.switch_to.frame(entry_iframe)

    expand_menu_and_scroll(driver)

    menu_list = []
    menu_info_list = []
    price_list = []

    menu_items = driver.find_elements(By.CSS_SELECTOR, "li.E2jtL")
    for it in menu_items:
        try: mn = it.find_element(By.CSS_SELECTOR, "span.lPzHi").text
        except: mn = ""

        try: mi = it.find_element(By.CSS_SELECTOR, "div.kPogF").text
        except: mi = ""

        try: pr = it.find_element(By.CSS_SELECTOR, "div.GXS1X em").text
        except: pr = ""

        menu_list.append(mn)
        menu_info_list.append(mi)
        price_list.append(pr)

    print(f"📌 메뉴 {len(menu_list)}개")

    return store_id, name, name_type, location, locinfo, info, menu_list, menu_info_list, price_list


# ============================================================
# 🔥 8. 메인 루프
# ============================================================
START_PAGE = 4
END_PAGE = 5

for page in range(START_PAGE, END_PAGE + 1):

    print("\n===================================")
    print(f"🚀 {page} 페이지 시작")
    print("===================================")

    if page != 1:
        if not go_to_page(driver, page):
            break

    # 여기에서 전체 스크롤 후 안정적으로 item 재수집
    items = get_all_items_after_full_scroll(driver)

    for idx in range(len(items)):
        #skip_idx = set(range(43, 48))   # 43,44,45,46,47 제외

        #if idx in skip_idx:
        #    print(f"⏩ {idx}번째 가게는 의도적으로 스킵됨")
        #    continue

        print(f"➡ {idx+1}/{len(items)} 번째 가게 수집 중")

        # 상세 페이지 열기
        open_store_page(driver, idx)

        store_id, name, name_type, loc, locinfo, info, menus, menuinfo, prices = scrape_store_detail(driver)

        NameList.append(name)
        NametypeList.append(name_type)
        LocationList.append(loc)
        LocinfoList.append(locinfo)
        InfoList.append(info)
        MenuList.append(menus)
        MenuinfoList.append(menuinfo)
        PriceList.append(prices)

        count_saved += 1
        print(f"⭐ 저장 완료: {count_saved}개")

        # 다시 검색 목록 iframe으로 복귀
        driver.switch_to.default_content()
        driver.switch_to.frame(driver.find_element(By.CSS_SELECTOR, "iframe#searchIframe"))
        time.sleep(1)


# ============================================================
# 🔥 9. CSV 저장
# ============================================================
df = pd.DataFrame({
    "name": NameList,
    "type": NametypeList,
    "location": LocationList,
    "locinfo": LocinfoList,
    "info": InfoList,
    "menus": MenuList,
    "menu_info": MenuinfoList,
    "price": PriceList
})

df.to_csv("naver_data.csv", encoding="utf-8-sig", index=False)
print("🎉 리뷰 제외 전체 페이지 크롤링 완료! → naver_data.csv 저장됨")


🚀 4 페이지 시작
➡ 4 페이지 이동
📌 PageDown 방식으로 전체 스크롤 시작…
📌 더 이상 새로운 가게가 로딩되지 않아 스크롤 중단
📌 PageDown 스크롤 완료. 총 로딩된 가게 수: 70
📌 최종 수집 가능한 가게 수: 70
➡ 1/70 번째 가게 수집 중
📌 상세 iframe 입장 완료
📌 메뉴 전체 로딩 완료
📌 메뉴 9개
⭐ 저장 완료: 1개
➡ 2/70 번째 가게 수집 중
📌 상세 iframe 입장 완료
❌ '홈' 탭 클릭 실패
📌 메뉴 전체 로딩 완료
📌 메뉴 2개
⭐ 저장 완료: 2개
➡ 3/70 번째 가게 수집 중
📌 상세 iframe 입장 완료
📌 메뉴 전체 로딩 완료
📌 메뉴 16개
⭐ 저장 완료: 3개
➡ 4/70 번째 가게 수집 중
📌 상세 iframe 입장 완료
📌 메뉴 전체 로딩 완료
📌 메뉴 1개
⭐ 저장 완료: 4개
➡ 5/70 번째 가게 수집 중
📌 상세 iframe 입장 완료
📌 메뉴 전체 로딩 완료
📌 메뉴 1개
⭐ 저장 완료: 5개
➡ 6/70 번째 가게 수집 중
📌 상세 iframe 입장 완료
📌 메뉴 전체 로딩 완료
📌 메뉴 8개
⭐ 저장 완료: 6개
➡ 7/70 번째 가게 수집 중
📌 상세 iframe 입장 완료
📌 메뉴 전체 로딩 완료
📌 메뉴 8개
⭐ 저장 완료: 7개
➡ 8/70 번째 가게 수집 중
📌 상세 iframe 입장 완료
📌 메뉴 전체 로딩 완료
📌 메뉴 32개
⭐ 저장 완료: 8개
➡ 9/70 번째 가게 수집 중
📌 상세 iframe 입장 완료
📌 메뉴 전체 로딩 완료
📌 메뉴 32개
⭐ 저장 완료: 9개
➡ 10/70 번째 가게 수집 중
📌 상세 iframe 입장 완료
📌 메뉴 전체 로딩 완료
📌 메뉴 20개
⭐ 저장 완료: 10개
➡ 11/70 번째 가게 수집 중
📌 상세 iframe 입장 완료
📌 메뉴 전체 로딩 완료
📌 메뉴 20개
⭐ 저장 완료: 11개
➡ 12/70 번째 가게 수집 중
📌 상세 iframe 입장 완료
📌 메뉴 전체 로딩 완료
📌 메뉴 7개
⭐ 저장 완료: 12개


In [8]:
# ============================================================
# 🔥 9. CSV 저장
# ============================================================
df = pd.DataFrame({
    "name": NameList,
    "type": NametypeList,
    "location": LocationList,
    "locinfo": LocinfoList,
    "info": InfoList,
    "menus": MenuList,
    "menu_info": MenuinfoList,
    "price": PriceList
})

df.to_csv("naver_data.csv", encoding="utf-8-sig", index=False)
print("🎉 리뷰 제외 전체 페이지 크롤링 완료! → naver_data.csv 저장됨")

🎉 리뷰 제외 전체 페이지 크롤링 완료! → naver_crawling_no_review.csv 저장됨
